In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('news_dataset.csv',encoding='iso-8859-1')
print(data.head())

      id               author        date  year month         topic  \
0  17307       Marlise Simons   1/01/2017  2017     1  architecture   
1  17292          Andy Newman  31/12/2016  2016    12           art   
2  17298  Emma G. Fitzsimmons   2/01/2017  2017     1      business   
3  17311           Carl Hulse   3/01/2017  2017     1      business   
4  17339        Jim Rutenberg   5/01/2017  2017     1      business   

                                             article  
0  PARIS  ?   When the Islamic State was about to...  
1  Angels are everywhere in the Mu?iz family?s ap...  
2  Finally. The Second Avenue subway opened in Ne...  
3  WASHINGTON  ?   It?s   or   time for Republica...  
4  For Megyn Kelly, the shift from Fox News to NB...  


In [3]:
topic_counts = data['topic'].value_counts()

# Print the count of every topic
print("Topic Counts:")
print(topic_counts)

Topic Counts:
politics         324
business         208
entertainment    153
crime            110
lifestyle         78
law               41
sports            30
science           25
technology        18
architecture       4
accidents          4
art                2
health             2
environment        1
Name: topic, dtype: int64


In [4]:
def process_document(document):
    # Split the document into words
    words = document.split()

    # Check if the first word is all uppercase
    if words[0].isupper():
        # Find the index of the first question mark if it exists
        try:
            first_question_index = words.index('?')
        except ValueError:
            # '?' not found, set first_question_index to the length of the list
            first_question_index = len(words)

        # Remove words until the first question mark (inclusive)
        words = words[first_question_index:]

    # Join the remaining words back into a string
    result = ' '.join(words)
    # Check if the first character of the result is not '?'
    if result and result[0] != '?':
        return result
    else:
        # Return the result without the first character if it starts with '?'
        return result[1:]


# Assuming data is a DataFrame containing articles in a column named 'article'
# Apply process_document to all documents in the 'article' column
data['processed_article'] = data['article'].apply(process_document)

In [5]:
data['processed_article'] = data['processed_article'].str.replace("?", "'")

/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [6]:
data['processed_article'][0]

" When the Islamic State was about to be driven out of the ancient city of Palmyra in March, Yves Ubelmann got a call from Syria's director of antiquities to come over in a hurry. An architect by training, Mr. Ubelmann, 36, had worked in Syria before the country was engulfed by war. But now there was special urgency for the kind of work his youthful team of architects, mathematicians and designers did from their cramped offices in Paris: producing digital copies of threatened historical sites. Palmyra, parts of it already destroyed by the Islamists who deemed these monuments idolatrous, was still rigged with explosives. So he and Houmam Saad, his Syrian colleague, spent four days flying a drone with a robot camera over the crumbled arches and temples. 'Drones with four or six rotors can hover really close and register structural details, every crack and hole, and we can take very precise measurements,' said Mr. Ubelmann, who founded the company Iconem. 'This is the stuff architects and

In [7]:
entertainment_data = data[data['topic'] == 'entertainment']
entertainment_subset = entertainment_data[['id', 'processed_article']]
entertainment_subset

,id,processed_article
19,17285,"When Walt Disney's 'Bambi' opened in 1942, cri..."
20,17286,"Death may be the great equalizer, but it isn't..."
21,17297,It's the season for family travel and photos '...
22,17306,Mariah Carey suffered through a performance tr...
23,17308,Pop music and fashion never met cuter than in ...
...,...,...
931,18386,Senator Jeff Sessions was confirmed on Wednes...
951,18408,"Was it just a year ago that Katie Holmes, Jenn..."
956,18413,Mother Jones was named magazine of the year on...
959,18417,"Damien Chazelle, the writer and director of th..."


In [8]:
import spacy
import neuralcoref
nlp=spacy.load('en')


In [9]:
neuralcoref.add_to_pipe(nlp)

In [10]:
# Assuming 'data' is your DataFrame
test = data.loc[data['id'] == 17307, 'processed_article'].values[0]

In [11]:
doc = nlp(test)

doc._.has_coref
coref_clusters=doc._.coref_clusters
coref_clusters

[Syria: [Syria, Syria, the country],
 his: [his, he, his],
 mathematicians and designers: [mathematicians and designers, their],
 Palmyra: [Palmyra, it],
 This: [This, it],
 the stuff architects and archaeologists need: [the stuff architects and archaeologists need, They],
 Mr. Ubelmann: [Mr. Ubelmann, Mr. Ubelmann, he],
 Palmyra: [Palmyra, Palmyra, Palmyra, Palmyra],
 The terrorists: [The terrorists, them],
 the best response: [the best response, It, the show, it, it, the show, it, the multimedia show, it],
 these places: [these places, their, their],
 a war of images: [a war of images, that war],
 he: [he, his, he, He],
 global heritage: [global heritage, the Middle Eastern heritage],
 Mr. Ubelmann: [Mr. Ubelmann, he, he, his],
 We: [We, We, our],
 the archaeologists: [the archaeologists, the Syrian archaeologists],
 They: [They, their, their],
 our colleagues: [our colleagues, they],
 his team: [his team, The team],
 Islamists: [Islamists, their],
 Nimrud: [Nimrud, she],
 specialist

In [12]:
def getArticle(number):
     return data.loc[data['id'] ==number, 'processed_article'].values[0]

In [13]:
class create_cluster():
    nlp = spacy.load('en_core_web_sm')
    neuralcoref.add_to_pipe(nlp)

    coref_clusters = None  # Define coref_clusters at the class level

    @staticmethod
    def cluster(context):
        doc = create_cluster.nlp(context)  # Access nlp using class name
        if doc._.has_coref:
            create_cluster.coref_clusters = doc._.coref_clusters  # Store clusters in class attribute

    @staticmethod
    def getClusters():
        return create_cluster.coref_clusters  # Access coref_clusters using class name

In [14]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
def extract_relevant_sentences_hf(context, question,num_context_sentences=0):
    model_name = "paraphrase-MiniLM-L6-v2"  
    model = SentenceTransformer(model_name)
    create_cluster.cluster(context)
    context_sentences = nltk.sent_tokenize(context)
    # Encode the question and context
    question_embedding = model.encode(question, convert_to_tensor=True)
    context_embeddings = model.encode(context_sentences, convert_to_tensor=True)

    # Compute cosine similarity between question and context sentences
    cos_similarities = util.pytorch_cos_sim(question_embedding, context_embeddings)[0]

    cos_similarities = util.pytorch_cos_sim(question_embedding, context_embeddings)[0]

    # Find the index of the sentence with the maximum similarity
    max_similarity_index = cos_similarities.argmax().item()

    start_index = max(0, max_similarity_index - num_context_sentences)
    end_index = min(len(context_sentences) - 1, max_similarity_index + num_context_sentences)

    context_paragraph = ' '.join(context_sentences[start_index:end_index + 1])

    return context_paragraph #cos_similarities[max_similarity_index].item()
def extract_relevant_sentences_tfidf(context, question, num_context_sentences=3):
    # Tokenize context into sentences
    create_cluster.cluster(context)
    context_sentences = nltk.sent_tokenize(context)

    # Tokenize question into words
    question_words = nltk.word_tokenize(question.lower())

    # Vectorize context sentences using TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(context_sentences)

    # Calculate TF-IDF vector for the question
    question_tfidf = tfidf_vectorizer.transform([' '.join(question_words)])

    # Calculate cosine similarity between question TF-IDF vector and context TF-IDF matrix
    cosine_similarities = cosine_similarity(question_tfidf, tfidf_matrix).flatten()

    # Find the indices of top N most similar sentences
    top_indices = cosine_similarities.argsort()[-num_context_sentences:][::-1]

    # Extract relevant sentences
    relevant_paragraph = ' '.join([context_sentences[i] for i in top_indices])

    return relevant_paragraph




In [15]:
import spacy
from nltk.tokenize import word_tokenize
from nltk import pos_tag

money_words = [
    'dollar', 'euro', 'pound', 'yen', 'payment', 'price', 'cost', 'fee', 'salary',
    'profit', 'revenue', 'income', 'expenditure', 'budget',
    'bank', 'account', 'credit', 'loan', 'interest', 'GDP', 'inflation', 'deflation',
    'exchange rate', 'stock', 'bond', 'dividend', 'capital', 'tax', 'duty', 'levy',
    'refund', 'revenue', 'expense', 'loss', 'cash', 'credit card', 'check', 'PayPal','amount'
]
date_related_words = [
    'celebration', 'anniversary', 'ceremony', 'festival', 'holiday', 'occasion', 'commemoration',
    'era', 'century', 'decade', 'generation', 'period', 'epoch', 'age',
    'timeline', 'chronology', 'schedule', 'agenda', 'timetable', 'deadline', 'due date', 'expiry date',
    'duration', 'timeframe', 'interval', 'period', 'length', 'span', 'term', 'periodicity',
    'yesterday', 'today', 'tomorrow', 'past', 'present', 'future', 'then', 'now',
    'spring', 'summer', 'autumn', 'winter', 'monsoon', 'rainy season', 'dry season', 'harvest season',
    'date', 'day', 'week', 'month', 'year', 'decade', 'century', 'millennium',
    'age', 'youth', 'aging', 'seniority', 'life stage'
]
person_question_words = ['who', 'whose', 'whom', 'name', 'identity', 'character', 'individual', 'subject', 'figure', 'protagonist']

gpe_question_words = ['where', 'location', 'place', 'country', 'city', 'state', 'region', 'province', 'town', 'area','capital']

def answertype(article, question):

    context=extract_relevant_sentences_tfidf(article,question)
    nlp = spacy.load('en_core_web_sm')
    
    if questiontype(question):
        t = 'DESCRIPTIVE'
        word = word_tokenize(question.lower())
        if any(x in word for x in person_question_words):
            t = 'PERSON'
        elif 'where' in word:
            t = 'GPE'
        elif any(x in word for x in date_related_words):
            t = 'DATE'
        elif 'when' in word or any(x in word for x in date_related_words):
            t = 'DATE'
        elif any(x in word for x in money_words):
            t= 'MONEY'
        
        coref_clusters=create_cluster.getClusters()
        # Perform further processing based on answer type
        if t == 'DESCRIPTIVE':
            # Perform descriptive processing
            # For now, return the question itself
            return question
        
        else:
            # Perform processing based on entity extraction
            # Extract entities of the identified answer type from the context
            
            
            doc = nlp(context)
            
            entities = [ent.text for ent in doc.ents if ent.label_ == t]
            #events = extract_events(doc)  # Extract events from the context
            #relations = extract_relations(doc)  # Extract relations between entities
            
            if coref_clusters:
                # Check if the first entity is in any head cluster
                first_entity = entities[0] if entities else None
                for cluster in coref_clusters:
                    if first_entity and first_entity in cluster.main.text:
                        return cluster.main.text  # Return the head of the coreference cluster

            # If entities of the answer type are found, return the first one
            if entities:
                return entities[0]
            #elif events:
              #  return events[0]
           # elif relations:
               # return relations[0]
            else:
                # If no entities, events, or relations are found, return the context itself
                return "Couldn't find the answer in the context."

    else:
        # Perform descriptive processing
        # For now, return the question itself
        return "Not A Question!"


def questiontype(question):
    # Define question tags
    questiontags = ['WP', 'WDT', 'WP$', 'WRB']

    # Tokenize and tag the question
    question_POS = pos_tag(word_tokenize(question.lower()))
    
    # Identify question tags
    question_tags = [token for token, tag in question_POS if tag in questiontags]

    # Check if there is only one question tag and it's not "what"
    if len(question_tags) == 1 :
        return True
    else:
        return False

#def extract_events(doc):
 #   events = []
  #  for token in doc:
   #     if token.pos_ == "VERB":
    #        events.append(token.text)
    #return events

#def extract_relations(doc):
 #   relations = []
  #  for token in doc:
   #     if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
    #        # Extract relation in the form of subject-verb
     #       relation = f"{token.text}-{token.head.text}"
      #      relations.append(relation)
    #return relations


In [16]:

# Example relevant context and question
question = "Who founded the company Iconem"
relevant=extract_relevant_sentences_tfidf(test,question)
#print(f"Most similar sentence: {relevant}")
#print(f"Similarity score: ")

# Extract the answer
answer = answertype(getArticle(17307), question)
print("Answer:", answer)


Answer: Mr. Ubelmann


In [17]:
test_questions=pd.read_csv("new_questions.csv")

In [18]:
from sklearn.metrics import f1_score

# Initialize lists to store expected and extracted answers
expected_answers = []
extracted_answers = []

for index, row in test_questions.iterrows():
    question = row['question']
    article_id = row['article']
    expected_answer = row['answer']
    
    # Fetch the article based on the article_id
    test_article = data.loc[data['id'] == article_id, 'processed_article'].values[0]
    
    # Extract relevant sentences from the article
    relevant_sentences = extract_relevant_sentences_hf(test_article, question)
    
    # Extract the answer
    answer = answertype(relevant_sentences, question)
    
    # Append expected and extracted answers to the lists
    expected_answers.append(expected_answer)
    print(expected_answer)
    extracted_answers.append(answer)
    print(answer)

# Compute the precision score
precision = f1_score(expected_answers, extracted_answers, average='micro')

print("Precision Score:", precision)


Yves Ubelmann
Mr. Ubelmann
Martinez
Louvre
Houmam Saad
Ubelmann
President François Hollande
Palmyra
President François Hollande
Couldn't find the answer in the context.
Yves Ubelmann
Couldn't find the answer in the context.
Cheikhmous Ali
Cheikhmous Ali
Cheikhmous Ali
Cheikhmous Ali
President François Hollande
Hollande
France
Couldn't find the answer in the context.
Zoraida Mu'iz and her family
Couldn't find the answer in the context.
Zoraida Mu'iz
Not A Question!
Jos' Mu'iz
Ms. Mu'iz
Jos' Jr.
Jos' Jr.
Jesus
Jesus
Jos' Mu'iz
Couldn't find the answer in the context.
Surgeons
Mr. Mu'iz
Jesus
Jesus
Zoraida Mu'iz
Ms. Mu'iz
Maria
Couldn't find the answer in the context.
Precision Score: 0.25


In [19]:
expected_answers = []
extracted_answers = []

for index, row in test_questions.iterrows():
    question = row['question']
    article_id = row['article']
    expected_answer = row['answer']
    
    # Fetch the article based on the article_id
    test_article = data.loc[data['id'] == article_id, 'processed_article'].values[0]
    
    # Extract relevant sentences from the article
    relevant_sentences = extract_relevant_sentences_tfidf(test_article, question)
    
    # Extract the answer
    answer = answertype(relevant_sentences, question)
    
    # Append expected and extracted answers to the lists
    expected_answers.append(expected_answer)
    print(expected_answer)
    extracted_answers.append(answer)
    print(answer)

# Compute the precision score
precision = f1_score(expected_answers, extracted_answers, average='micro')

print("Precision Score:", precision)

Yves Ubelmann
Ubelmann
Martinez
Martinez
Houmam Saad
Cheikhmous Ali
President François Hollande
Palmyra
President François Hollande
Couldn't find the answer in the context.
Yves Ubelmann
Nineveh
Cheikhmous Ali
Cheikhmous Ali
Cheikhmous Ali
Cheikhmous Ali
President François Hollande
Mr. Hollande
France
Palmyra
Zoraida Mu'iz and her family
Dickens
Zoraida Mu'iz
Not A Question!
Jos' Mu'iz
Mu'iz
Jos' Jr.
Jesus
Jesus
Jesus
Jos' Mu'iz
Ms. Mu'iz
Surgeons
Mr. Mu'iz
Jesus
Jesus
Zoraida Mu'iz
Mu'iz
Maria
Maria
Precision Score: 0.3


In [20]:
import json

# Assuming squad_test["data"] contains the JSON data
with open("SQUAD-Testing.json", "r") as f:
    squad_test = json.load(f)

# Iterate through the "data" array
for data_item in squad_test["data"]:
    # Read the context
    context = data_item["context"]
    print("Context:", context)

    # Iterate through the questions
    for question_item in data_item["questions"]:
        question = question_item["question"]
        print("Question:", question)
        predicted_answer=answertype(context,question)
        # You can also access the plausible answers if needed
        plausible_answers = question_item["plausible_answers"]
        for answer in plausible_answers:
            answer_text = answer["text"]
            print("Predicted Answer:",predicted_answer)
            print("Plausible Answer:", answer_text)
        


Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
Question: In what country is Normandy located?
Predicted Answer: In what country is Normandy located?
Plausible Answer: France
Question: When were the Normans in Normandy?
Predicted Answer: the first half of the 10th century
Plausible Answer: 10th 

In [21]:
import ipywidgets as widgets
from IPython.display import display,clear_output

# Define input widgets
question_input = widgets.Text(description='Question:', placeholder='Enter your question', disabled=False)
article_input = widgets.IntText(description='Article Number:', value=0, disabled=False)
answer_output=widgets.Output(description="Answer")
# Define a function to handle user input and display the answer


# Define a callback function to be called when the button is clicked
def on_button_click(b):
    question = question_input.value
    article_number = article_input.value
    if question and article_number:
        article=getArticle(article_number)
    else:
        print("Give Proper Questiona and Article")
    with answer_output:
        clear_output(wait=True)
    
    # Display the answer
    with answer_output:
        answer = answertype(article, question)
        print("Answer:",answer)

# Create a button widget
button = widgets.Button(description='Get Answer')
button.on_click(on_button_click)

# Display the input widgets and button
display(question_input, article_input,answer_output, button)


Text(value='', description='Question:', placeholder='Enter your question')

IntText(value=0, description='Article Number:')

Output()

Button(description='Get Answer', style=ButtonStyle())

In [22]:
import json

# Assuming squad_test["data"] contains the JSON data
with open("SQUAD-Testing.json", "r") as f:
    squad_test = json.load(f)

# Iterate through the "data" array
for data_item in squad_test["data"]:
    # Read the context
    context = data_item["context"]
    print("Context:", context)

    # Iterate through the questions
    for question_item in data_item["questions"]:
        question = question_item["question"]
        print("Question:", question)
        predicted_answer=answertype(context,question)
        # You can also access the plausible answers if needed
        plausible_answers = question_item["plausible_answers"]
        for answer in plausible_answers:
            answer_text = answer["text"]
            print("Predicted Answer:",predicted_answer)
            print("Plausible Answer:", answer_text)
        


Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
Question: In what country is Normandy located?
Predicted Answer: In what country is Normandy located?
Plausible Answer: France
Question: When were the Normans in Normandy?
Predicted Answer: the first half of the 10th century
Plausible Answer: 10th 